## Creating and managing disks (External Hard drive)

This can be done via the **disk** option in the **compute engine** interface in **google cloud console**

**IMPORTANT**: The **Zone of the disk and the zone of VM Instance must be same!**

### 1) Creating disk with [gcloud compute disks](https://cloud.google.com/sdk/gcloud/reference/compute/disks/)

> **[create](https://cloud.google.com/sdk/gcloud/reference/compute/disks/create) a persistent disk**
```bash
gcloud compute disks create [DISK_NAME] --size=[SIZE]GB --zone=[ZONE | DEFAULT IF UNSPECIFIED]
```
> **New disk must be formatted before being used**

### 2) Attaching disk to VM instance with [gcloud compute instances attach-disk](https://cloud.google.com/sdk/gcloud/reference/compute/instances/attach-disk)

> **Attach disk to instance** (must be located in same zone)
```bash
gcloud compute instances attach-disk [VM_INSTANCE_NAME] --disk [DISK_NAME]
```

> To **check if the disk is attached** to the instance, **log in** and use the command **lsblk**
```bash
gcloud compute ssh [USERNAME]@[INSTANCE_NAME]
lsblk
```
>> **output :**
```bash
NAME   MAJ:MIN RM  SIZE RO TYPE MOUNTPOINT
sda      8:0    0   10G  0 disk 
`-sda1   8:1    0   10G  0 part /
sdb      8:16   0  100G  0 disk (This is the new disk)
```


### 3) [Format and mount disk](https://cloud.google.com/compute/docs/disks/add-persistent-disk#formatting)

> **Format**: (DISK_ID is **sdb**)
```bash
sudo mkfs.ext4 -m 0 -F -E lazy_itable_init=0,lazy_journal_init=0,discard /dev/[DISK_ID]
```

> Create a **directory to mount disk**
```bash
sudo mkdir -p /mnt/disks/new_disk
```
> **Mount disk**
```bash
sudo mount -o discard,defaults /dev/[DISK_ID] /mnt/disks/new_disk
```
> Give **read and write permission**
```bash
sudo chmod a+w /mnt/disks/new_disk
```
> **check mount point**
```bash
lsblk
```
>>```bash
NAME   MAJ:MIN RM  SIZE RO TYPE MOUNTPOINT
sda      8:0    0   10G  0 disk 
`-sda1   8:1    0   10G  0 part /
sdb      8:16   0  100G  0 disk /mnt/disks/new_disk
```

> **IMPORTANT:** The Disk will be **unmounted** when the instance is **stopped** and **started again**! 

> To avoid mounting process every time the instance is restarted, we have to edit the **/etc/fstab/** file

>>```bash
sudo cp /etc/fstab /etc/fstab.backup (create backup)
sudo blkid /dev/[DISK_ID] (Get UUID of disk)
```
>>> **out:** /dev/sdb: UUID="[UUID_FOR_DISK]" TYPE="ext4"

>>```bash
sudo vim /etc/fstab 
```
>>> Add the following line
>>> **UUID=[UUID_FOR_DISK] /mnt/disks/new_disk ext4 discard,defaults,nofail 0 2**

>>> **NOTE**: When the disk is **detached**, this line should be **removed**!


### 4) Deleting disk

> Before deleting the disk it has to be **detached** from the **instance**
```bash
gcloud compute instances detach-disk [INSTANCE_NAME] --disk [DISK_NAME]
gcloud compute disks delete [DISK_NAME]
```